In [ ]:
# File: belt.ipynb
# Code: Claude Code and Codex
# Review: Ryoichi Ando (ryoichi.ando@zozo.com)
# License: Apache v2.0

In [ ]:
import random

import numpy as np
from frontend import App

# create an app
app = App.create("belt")

# create a cylinder mesh for rollers
V, F = app.mesh.cylinder(r=0.5, min_x=-0.5, max_x=0.5, n=21)
app.asset.add.tri("cylinder", V, F)

# create 3 belt meshes with decreasing sizes
N, Nb, dirs = 3, 3, []
for k in range(Nb):
    r, hw = 1.25 + 2e-3 * k, 0.4 - 0.05 * k
    n = int(44 * hw)
    V, F = app.mesh.cylinder(r=r, min_x=-hw, max_x=hw, n=n)
    app.asset.add.tri(f"belt-{k}", V, F)

# arrange rollers in triangular configuration
for i in range(N):
    r, t = 0.65, 2 * i * np.pi / N
    x, y = r * np.sin(t), r * np.cos(t)
    dirs.append([x, y])

# create a scene
scene = app.scene.create()

# add rollers with spinning and moving animations
for x, y in dirs:
    s, w, axis = 0.511, 60, [0, 0, 1]
    target = [s * x, s * y, 0]
    jitter = 1e-2 * random.random()
    # add roller rotated to be horizontal
    obj = scene.add("cylinder").rotate(90, "y").at(x, y, jitter)
    obj.param.set("friction", 0.5)
    pin = obj.pin()
    # spin the roller and move it to target position
    pin.spin(t_start=1.5, t_end=3, angular_velocity=w, axis=axis).move_by(
        target, t_start=0.0, t_end=1.0
    )

# add belts with strain limiting
for k in range(Nb):
    obj = scene.add(f"belt-{k}").rotate(90, "y")
    obj.param.set("strain-limit", 0.05)

# set preview options
opts = {"eyeup": 0.7, "pin": False, "wireframe": True}

# compile the scene and report stats
scene = scene.build().report()

# preview the initial scene
scene.preview(options=opts)

In [ ]:
# create a new session with the compiled scene
session = app.session.create(scene)

# set session parameters
(
    session.param.set("dt", 0.01)
    .set("frames", 200)
    .set("gravity", 0.0)
    .set("min-newton-steps", 32)
)

# build this session
session = session.build()

In [ ]:
# start the simulation and live-preview the results
session.start().preview(options=opts)

# also show simulation logs in realtime
session.stream()

In [ ]:
# create an animation from the simulation results
session.animate(options=opts)

In [ ]:
# export the animation to file
session.export.animation()

In [ ]:
# this is for CI
if app.ci:
    assert session.finished()